In [6]:
import os
import torch
from torch.utils.data import DataLoader
from model_paired_ae_emma import MultimodalAE, Encoder, Decoder
from src.dataset import generate_datasets, concat_datasets
import torch.nn as nn

### Data

In [2]:
# Génération des ensembles de données
train_datasets = generate_datasets(suffix='5_diff', type='paired', train=True, test=False)
test_datasets = generate_datasets(suffix='5_diff', type='paired', train=False, test=True)


# Création des chargeurs de données
train_loaders = [DataLoader(dataset, batch_size=32, shuffle=True) for dataset in train_datasets]
test_loaders = [DataLoader(dataset, batch_size=32, shuffle=False) for dataset in test_datasets]


# Obtenez les dimensions d'entrée à partir des ensembles de données
n_inputs1 = train_datasets[0][0][0].shape[0]  # La taille du vecteur de caractéristiques pour la modalité 1
n_inputs2 = train_datasets[1][0][0].shape[0]  # La taille du vecteur de caractéristiques pour la modalité 2
n_inputs3 = train_datasets[2][0][0].shape[0]  # La taille du vecteur de caractéristiques pour la modalité 3

Loading paired dataset
Loading paired dataset


### Initialisation des modèles

In [3]:
latent_dims = 20   
n_hiddens = 256

# Création du modèle
encoder1 = Encoder(n_inputs1, latent_dims, n_hiddens)
encoder2 = Encoder(n_inputs2, latent_dims, n_hiddens)
encoder3 = Encoder(n_inputs3, latent_dims, n_hiddens)

z1 = encoder1(train_datasets[0][0][0])
z2 = encoder2(train_datasets[1][0][0])
z3 = encoder3(train_datasets[2][0][0])

decoder1 = Decoder(n_inputs1, latent_dims, n_hiddens)
decoder2 = Decoder(n_inputs2, latent_dims, n_hiddens)
decoder3 = Decoder(n_inputs3, latent_dims, n_hiddens)

model1 = MultimodalAE(encoder=encoder1, decoder=decoder1)
model2 = MultimodalAE(encoder=encoder2, decoder=decoder2)
model1_2 = MultimodalAE(encoder=encoder1, decoder=decoder2)
model2_1 = MultimodalAE(encoder=encoder2, decoder=decoder1)
model3 = MultimodalAE(encoder=encoder3, decoder=decoder3)
model1_3 = MultimodalAE(encoder=encoder1, decoder=decoder3)
model3_1 = MultimodalAE(encoder=encoder3, decoder=decoder1)
model2_3 = MultimodalAE(encoder=encoder2, decoder=decoder3)
model3_2 = MultimodalAE(encoder=encoder3, decoder=decoder2)

### Critère et optimiseur

In [4]:
critere = nn.MSELoss()
lr = 0.001
optimizer1 = torch.optim.Adam(model1.parameters(), lr=lr)
optimizer2 = torch.optim.Adam(model2.parameters(), lr=lr)
optimizer1_2 = torch.optim.Adam(model1_2.parameters(), lr=lr)
optimizer2_1 = torch.optim.Adam(model2_1.parameters(), lr=lr)
optimizer3 = torch.optim.Adam(model3.parameters(), lr=lr)
optimizer1_3 = torch.optim.Adam(model1_3.parameters(), lr=lr)
optimizer3_1 = torch.optim.Adam(model3_1.parameters(), lr=lr)

### Train

In [5]:
num_epochs = 100
for epoch in range(num_epochs):
    model1.train()
    model3.train()
    model1_3.train()
    model3_1.train()
    for (x1, _), (x2, _), (x3, y) in zip(*train_loaders):

        if x1.size(0) != 32 or x3.size(0) != 32 or y.size(0) != 32:
            continue

       # Forward pass
        o1, o3, o1_3, o3_1 = model1(x1), model3(x3), model1_3(x1), model3_1(x3)
        # loss = critere(o1, x1) + critere(o3, x3) + 0*critere(o1_3, x3) + 0*critere(o3_1, x1)
        loss = critere(o1, x1) + critere(o3, x3) + critere(o1_3, x3) + critere(o3_1, x1)
        
        # Backward and optimize
        optimizer1.zero_grad()
        optimizer3.zero_grad()
        # optimizer1_3.zero_grad()
        # optimizer3_1.zero_grad()

        loss.backward()

        optimizer1.step()
        optimizer3.step()
        # optimizer1_3.step()
        # optimizer3_1.step()
        

    print(f' Epoch {epoch} -> loss totale :', round(loss.item(), 4), " loss1 :", round(critere(o1, x1).item(), 4), " loss3 :", round(critere(o3, x3).item(), 4), " loss1_3 :", round(critere(o1_3, x3).item(), 4), " loss3_1 :", round(critere(o3_1, x1).item(), 4))


 Epoch 0 -> loss totale : 0.872  loss1 : 0.4587  loss3 : 0.4133  loss1_3 : 1.1662  loss3_1 : 1.2347
 Epoch 1 -> loss totale : 0.7156  loss1 : 0.4271  loss3 : 0.2885  loss1_3 : 1.1052  loss3_1 : 1.2175
 Epoch 2 -> loss totale : 0.6793  loss1 : 0.3718  loss3 : 0.3074  loss1_3 : 1.0663  loss3_1 : 1.1322
 Epoch 3 -> loss totale : 0.5986  loss1 : 0.3565  loss3 : 0.2421  loss1_3 : 1.0444  loss3_1 : 1.1044
 Epoch 4 -> loss totale : 0.5453  loss1 : 0.2999  loss3 : 0.2454  loss1_3 : 1.126  loss3_1 : 1.2976
 Epoch 5 -> loss totale : 0.4965  loss1 : 0.285  loss3 : 0.2115  loss1_3 : 1.1227  loss3_1 : 1.1562
 Epoch 6 -> loss totale : 0.4808  loss1 : 0.2808  loss3 : 0.2  loss1_3 : 1.1729  loss3_1 : 1.3684
 Epoch 7 -> loss totale : 0.4317  loss1 : 0.2535  loss3 : 0.1782  loss1_3 : 1.2115  loss3_1 : 1.4
 Epoch 8 -> loss totale : 0.4174  loss1 : 0.247  loss3 : 0.1703  loss1_3 : 1.2389  loss3_1 : 1.2847
 Epoch 9 -> loss totale : 0.4088  loss1 : 0.2453  loss3 : 0.1635  loss1_3 : 1.2331  loss3_1 : 1.3686


### Test

In [6]:
model1.eval()
model3.eval()
model1_3.eval()
model3_1.eval()

with torch.no_grad():
    total_loss = 0
    for (x1, _), (x2, _), (x3, y) in zip(*test_loaders):
        o1, o3, o1_3, o3_1 = model1(x1), model3(x3), model1_3(x1), model3_1(x3)
        loss = critere(o1, x1) + critere(o3, x3) + critere(o1_3, x3) + critere(o3_1, x1)
        total_loss += loss.item()
    print(f'Test Loss: {total_loss / len(test_loaders[0])}')
    print(" loss1 :", critere(o1, x1).item(), " loss3 :", critere(o3, x3).item(), " loss1_3 :", critere(o1_3, x3).item(), " loss3_1 :", critere(o3_1, x1).item())

Test Loss: 3.4732488904680525
 loss1 : 0.42336663603782654  loss3 : 0.9735544919967651  loss1_3 : 1.0054936408996582  loss3_1 : 1.0614235401153564
